In [6]:
import numpy as np
import dill as pk

import os

from astroquery.mpc import MPC

from astroquery.jplhorizons import Horizons
import time
from astropy.time import Time

import requests


In [36]:
def get_radius(name, names, radii):
    for i, cur_name in enumerate(names):
        if name in cur_name:
            return radii[i]
    print("No match found")
    return 0

In [2]:
with open('./pks/ast_stats.pk', 'rb') as f:
    ast_dir = pk.load(f)

In [8]:
if not os.path.exists("./pks/ast_masses.pk"):
    r = requests.get("https://ssd-api.jpl.nasa.gov/sbdb_query.api?fields=full_name,diameter" )
    data = r.json()["data"]
    with open("./pks/ast_masses.pk", "wb") as f:
        pk.dump(data, f)
else:
    with open("./pks/ast_masses.pk", "rb") as f:
        data = pk.load(f)


In [31]:
names = []
radii = []

for i in range(len(data)):
    if data[i][1] is None: continue
    if float(data[i][1]) < 50: continue
    names.append(str(data[i][0]))
    radii.append(float(data[i][1]))

In [32]:
names = np.array(names)
radii = np.array(radii)

# ASO Asteroids

In [43]:
var_090 = np.empty(len(ast_dir.keys()))
var_150 = np.empty(len(ast_dir.keys()))
var_220 = np.empty(len(ast_dir.keys()))

for i, ast in enumerate(ast_dir.keys()):
    var_090[i] = ast_dir[ast]["flux_090"][1]
    var_150[i] = ast_dir[ast]["flux_150"][1]
    var_220[i] = ast_dir[ast]["flux_220"][1]

In [44]:
act_depth_090 = np.median(var_090)
act_depth_150 = np.median(var_150)
act_depth_220 = np.median(var_220)

#From here: https://simonsobs.atlassian.net/wiki/spaces/ANA/pages/872153096/ASO+Point+Source+Sensitivity+Forecasts
aso_depth_090 = 0.41
aso_depth_150 = 0.54
aso_depth_220 = 1.38

weight = 0.9 #This factor is the amount of time asteroids spend in the SO footprint. Assuming even sky coverage that includes
           #the whole ecliptic, this is 1, i.e. on average a asteroid will be observed as often as any random fixed point on 
           #the sky. If the asteroid spends a significant amount of time out of the footprint, then this weight should be 
           #correspondingly lower. I've conservatively picked 0.9.

N_aso_090 = (act_depth_090/aso_depth_090)**(1.25)*weight*170
N_aso_150 = (act_depth_150/aso_depth_150)**(1.25)*weight*170
N_aso_220 = (act_depth_220/aso_depth_220)**(1.25)*weight*170



In [45]:
print(N_aso_090, N_aso_150, N_aso_220) #Number of detections

19430.186542258845 18301.924104047917 23267.78306441227


In [46]:
(act_depth_150/aso_depth_150)**(1.25)*weight #Factor increase

np.float64(107.6583770826348)

# ASO TNOs

In [68]:
name = 'Pallas'
r_ast = get_radius(name, names, radii)

d_ast = 2.36
d_sast = 2.36



t = Time(time.time(), format='unix', scale='utc')

#tno_name = "Haumea (primary body)"
#tno_name = "Eris (primary body)"
#tno_name = "Makemake"
#tno_name = "Pluto Barycenter"
#tno_name = "Gonggong"
#tno_name = "Quaoar (system barycenter)"
#tno_name = "Sedna"
tno_name = "Orcus (system barycenter)"
#tno_name = "90482   "

obj = Horizons(id=tno_name, location='568', epochs=t.jd)


d_tno = obj.ephemerides()["r"].value
d_stno = d_tno
radius = 450

geo_factor = (d_tno/d_ast)**(-2)*(d_stno/d_sast)**(-1/2) * (radius/r_ast)**2 #Factor to scale asteroid to TNO size, distance, etc
#noise_factor = np.sqrt((act_depth_090/aso_depth_090)**2+(act_depth_150/aso_depth_150)**2+(act_depth_220/aso_depth_220)**2)
noise_factor = act_depth_150/aso_depth_150
print(np.array(ast_dir[name]["tot_sn"])*geo_factor*noise_factor)

[3.2565619]
